In [10]:
import pandas as pd
import numpy as np
import anndata as ad

from PIL import Image

import os
import os.path as osp

In [20]:
DATA_DIR = "/home/alma/w-projects/ccf/data/ludlar-triangle/raw"
OUT_DIR = "/home/alma/w-projects/ccf/data/ludlar-triangle/curated"

In [24]:
crd

,x,y,pixel_x,pixel_y,sample
ATCTGTAATTGTACCC-1_8,57,15,630.616438,358.561644,7
GTTCCAGTCTGACCAT-1_8,53,17,605.479452,323.219178,7
ATGATCGGGAATAGAC-1_8,54,16,618.835616,332.739726,7
TTGGATTGGGTACCAC-1_8,55,17,603.835616,339.452055,7
TACCTCACGCTTGTAC-1_8,56,16,617.260274,348.972603,7
...,...,...,...,...,...
AGTCTCACAAGACTAC-1_8,30,54,99.931507,84.452055,7
GCGGAGAAACTTCGCA-1_8,33,55,83.356164,107.465753,7
GGCAAAGGCGCCAATA-1_8,34,54,96.712329,117.054795,7
TTCGGGCGCTAGTCTT-1_8,37,55,80.136986,140.068493,7


In [25]:
for ii in [1,2,3,5,6,7,8]:
    section = "section_{}".format(ii)
    df = pd.read_csv(os.path.join(DATA_DIR,section,"counts.tsv.gz"),
                      sep = "\t",
                      header = 0,
                      index_col =0).T
    crd = pd.read_csv(os.path.join(DATA_DIR,section,"coordinates.tsv"),
                      sep = "\t",
                      header = 0,
                      index_col = 0,
                     )
    
    crd = crd[["pixel_x","pixel_y"]].values

    var = pd.DataFrame(df.columns.values,
                       index = df.columns,
                       columns = ["gene"]
                      )
    
    scalefactors = dict(tissue_hires_scalef = 1,
                            spot_diameter_fullres = 320,
                           )

    obs = pd.DataFrame(df.index.values,
                               index = df.index,
                               columns = ["spot"],
                              )
    
    img = Image.open(osp.join(DATA_DIR,section,section + "_HE_cropped.jpg"))
    img = img.transpose(Image.FLIP_LEFT_RIGHT)
    img = img.transpose(Image.TRANSPOSE)
    img = img.transpose(Image.ROTATE_270)

    img = np.asarray(img)#.transpose(1,0,2)

    images = dict(hires = img)

    uns = dict(spatial = {section:dict(images=images,
                                  scalefactors = scalefactors,
                                 )})

    
    adata = ad.AnnData(df.values,
                           var = var,
                           obs = obs,
                           uns = uns
                          )
    adata.obsm["spatial"] = crd
        
    adata.write_h5ad(osp.join(OUT_DIR,section + ".h5ad"))

In [19]:
adata

AnnData object with n_obs × n_vars = 252 × 21490
    obs: 'spot'
    var: 'gene'
    uns: 'spatial'
    obsm: 'spatial'